<a href="https://colab.research.google.com/github/itspriiyanshu/SpringML/blob/main/QuarksGluons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==1.3.2

Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Successfully uninstalled scikit-learn-1.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 47.3 MB/s eta 0:00:00


# **Load Dataset & Preprocess**
We start by removing nil rows and convert given particle level features (p_T, Rapidity, Azimuthal Angle, PDG ID) to jet level by taking seperate means for given 4 features of all particles (135-nil) for each jet

In [ ]:
import pandas as pd
import numpy as np

data=np.load('/content/drive/MyDrive/Colab Notebooks/QG_jets.npz')
print(data.files)
inputs=data['X']
njets=100000
inputs=inputs[:njets]
inputs_filtered=[]

X=np.zeros((len(inputs),4))
for i in range(len(inputs)):
  mask = ~(inputs[i] == 0).all(axis=1)
  inputs_filtered=(inputs[i][mask])
  X[i][0]=np.mean(inputs_filtered[:,0])
  X[i][1]=np.mean(inputs_filtered[:,1])
  X[i][2]=np.mean(inputs_filtered[:,2])
  X[i][3]=np.mean(inputs_filtered[:,3])
inputs=X
# inputs=inputs_filtered

# print(inputs.size)
# print(inputs_filtered)
target=data['y']
target=target[:njets]
print(inputs)
# print(target)

['X', 'y']
[[ 2.78231806e+01 -1.77256379e-02  4.71509695e+00 -8.77777778e+00]
 [ 3.12828409e+01  1.66467043e-01  3.02062727e+00 -9.88235294e+00]
 [ 8.89669563e+00 -1.37702443e+00  4.87845109e-01 -6.43157895e+01]
 ...
 [ 3.42609492e+01  1.01293798e+00  4.56279076e+00 -1.03687500e+02]
 [ 6.20155106e+00  2.58253519e-01  3.87450781e+00 -1.13863636e+01]
 [ 1.36461232e+01  1.03797887e+00  1.37801151e+00  3.23250000e+01]]


# Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2,random_state=42)


Ran it to find suitable hyperparams for XGBoost

*AI Generated*

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import xgboost as xgb

# Define XGBoost model
model = xgb.XGBClassifier(objective='binary:logistic', n_estimators=100)

# Define the hyperparameter search space
param_dist = {
    'max_depth': randint(3, 7),
    'learning_rate': [0.05, 0.1, 0.15],
    'n_estimators': randint(50, 200),
    'subsample': [0.8, 0.9, 1],
    'colsample_bytree': [0.7, 0.8, 1],
    'min_child_weight': randint(1, 5),
    'gamma': [0, 1],
    'lambda': [0, 1, 10],
    'alpha': [0, 1]
}

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=50, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)
print("Best hyperparameters found: ", random_search.best_params_)
best_model = random_search.best_estimator_
accuracy = best_model.score(X_test, y_test)
print(f"Accuracy of the best model: {accuracy:.2f}")


Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best hyperparameters found:  {'alpha': 0, 'colsample_bytree': 0.7, 'gamma': 0, 'lambda': 10, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 171, 'subsample': 1}
Accuracy of the best model: 0.77


## Implement XGBoost Classifier



In [ ]:
from xgboost import XGBClassifier

# XGBoost model
model = XGBClassifier(n_estimators=171,learning_rate=0.05,max_depth=3,min_child_weight=3,subsample=1,random_state=42)
model.fit(X_train,y_train)
accuracy=model.score(X_test,y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.77065
